# 제주도 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [3]:
chromedriver = '../../chromedriver.exe'
# chromedriver = 'C:\\Users\\TECH-8\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram 에 접속하여 로그인 

In [4]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [5]:
with open('password.txt') as f:
    password = f.read(8)

In [6]:
email = 'ckiekim'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [7]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [8]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [9]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도맛집 으로 검색

In [10]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [11]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글 열기

In [11]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 4. 게시글 정보 가져오기

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'협찬제주애월맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주도 #제주여행'

In [14]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월맛집',
 '#애월카페',
 '#애월읍맛집',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#제주도카페',
 '#제주카페',
 '#외도맛집',
 '#하귀맛집',
 '#한림맛집',
 '#한림카페',
 '#애월해안도로맛집',
 '#애월가볼만한곳',
 '#제주공항맛집',
 '#제주공항근처맛집',
 '#제주도',
 '#제주여행']

In [15]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [16]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [17]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [18]:
row = [content, date, like, place, tags]
row

['협찬제주애월맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주도 #제주여행',
 '2021-08-18',
 0,
 '',
 ['#애월맛집',
  '#애월카페',
  '#애월읍맛집',
  '#제주애월맛집',
  '#제주도맛집',
  '#제주맛집',
  '#제주도카페',
  '#제주카페',
  '#외도맛집',
  '#하귀맛집',
  '#한림맛집',
  '#한림카페',
  '#애월해안도로맛집',
  '#애월가볼만한곳',
  '#제주공항맛집',
  '#제주공항근처맛집',
  '#제주도',
  '#제주여행']]

### 5. 다음 게시글로 이동

In [39]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [18]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

In [10]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [23]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [13]:
keyword = '제주도관광'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(1)

In [14]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [15]:
# 200개 정도가 한계인 것 같음
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)

KeyboardInterrupt: 

In [24]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags
0,#난산리다방 #가오픈마지막주. '이번주가 마지막 입니다.'. 제가 운영하는 곳에 구...,"[#난산리다방, #가오픈마지막주., #르꼬르동블루]",2021-08-19,471,난산리다방
1,_오늘의 블루보틀✨오픈시간 아주 조금 지나서 도착하니바로 입장하고 여유로움 :)블루...,[],2021-08-18,"2,148",블루보틀 제주
2,#청춘나길 📢 스냅 촬영 안내 및 가격 공지🎁상품안내: 1시간 2시간 4시간 (그 ...,[#청춘나길],2021-08-17,670,환상의 섬 Jeju Island
3,▪️배 안나온다고 체중이 늘지 않는다고쓸데없이 걱정했던 지난날을 반성하며 ㅋㅋ출산이...,"[#제주여행, #태교여행, #법환바다안정기시절.., #임신10개월, #만삭]",2021-08-19,ela___님 외 521명이 좋아합니,법환바다
4,42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐...,"[#신해바라기분식, #제주도맛집뜨끈한, #순두부찌개, #국밥맛집, #제주맛집, #제...",2021-08-18,"1,079",제주도


In [25]:
df.tail()

,content,date,like,place,tags
109,"ო̤̜어깨 깡패 😏8, 9월 예약 받고 있습니다 🤗문의는 프로필의 오픈 카톡으로 부...",[],2021-08-19,19,제주시청 근처
110,.막 찍어도 화보 같은 월정리 카페 콧수염고객님의 멋진 사진 감사합니다 🤳제주시 구...,"[#구좌읍카페, #함덕맛집, #제주, #제주도맛집, #월정리, #월정리카페콧수염, ...",2021-08-19,3,
111,사장님 알로뷰 …❤️,[],2021-08-19,43,One and only - 원앤온리 제주
112,#제주여행 #1일차#사진못찍음,"[#제주여행, #1일차, #사진못찍음]",2021-08-19,4,
113,"부랴부랴 택배 싸놓고,오후는 율이와 피크닉 🧺 수업준비와 주문제작건은 야근 당첨😭제...","[#연이네다락방, #엄만야근해야해, #제주피크닉, #풍경이다했다, #어린이집휴원, ...",2021-08-19,9,선흘리


In [26]:
df.to_csv('data/제주여행.csv', index=False)

In [16]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [25]:
driver.close()

### 4가지 CSV 파일의 중복을 제거한 후 통합 저장

In [32]:
for keyword in ['제주도맛집','제주맛집','제주도관광','제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    print(df.head())

                                             content  \
0  .#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...   
1  <광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 ...   
2  .< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...   
3  협찬제주애월맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-...   
4  .한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...   

                                                date        like  place tags  
0  ['#연동', '#모니카옛날통닭간판부터', '#제주맛집', '#제주도맛집', '#제...  2021-08-19  1,113  NaN  
1  ['#섭지코지한끼🍱오픈', '#섭지코지맛집', '#섭지코지', '#섭지코지카페', ...  2021-08-18      0  NaN  
2  ['#제주맛집', '#제주도맛집', '#제주공항근처맛집', '#제주시맛집', '#애...  2021-08-18  1,008  NaN  
3  ['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...  2021-08-18      0  NaN  
4  ['#문도지오름방목된', '#제주맛집', '#제주도맛집', '#제주공항근처맛집', ...  2021-08-17  1,304  NaN  
                                             content  \
0  .💙 구좌 "제주모랑" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...   
1  협찬제주 대표 썬셋 맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 :...   
2  42년 전통 도민 맛집. 태양을 

In [30]:
jeju_df.tail()

,content,date,like,place,tags
109,"ო̤̜어깨 깡패 😏8, 9월 예약 받고 있습니다 🤗문의는 프로필의 오픈 카톡으로 부...",[],2021-08-19,19,제주시청 근처
110,.막 찍어도 화보 같은 월정리 카페 콧수염고객님의 멋진 사진 감사합니다 🤳제주시 구...,"['#구좌읍카페', '#함덕맛집', '#제주', '#제주도맛집', '#월정리', '...",2021-08-19,3,NaN
111,사장님 알로뷰 …❤️,[],2021-08-19,43,One and only - 원앤온리 제주
112,#제주여행 #1일차#사진못찍음,"['#제주여행', '#1일차', '#사진못찍음']",2021-08-19,4,NaN
113,"부랴부랴 택배 싸놓고,오후는 율이와 피크닉 🧺 수업준비와 주문제작건은 야근 당첨😭제...","['#연이네다락방', '#엄만야근해야해', '#제주피크닉', '#풍경이다했다', '...",2021-08-19,9,선흘리
